In [ ]:
import numpy as np
from scipy import sparse
from matplotlib import pyplot as plt
import time
import codecs

In [ ]:
plt.rcParams['figure.figsize'] = (10, 5)

In [ ]:
def save_results_to_tbl(results, sizes, fn, caption):
    with codecs.open(f'tbl/{fn}.tex', 'w', 'utf-8') as out:
        # head
        out.write('\\begin{table}[H]\n')
        out.write('\t\\centering\n')

        # body
        out.write('\t\\begin{tabular}{|c||' + 'c|' * len(sizes) + '}' + '\\hline\n')
        out.write('\t\tРозмір задачі & ' + ' & '.join(map(str, sizes)) + ' \\\\ \\hline \\hline\n')
        for algo, result in results.items():
            out.write('\t\t' + algo + ' & ' +
                      ' & '.join(map(lambda r: f'{r:.2f}'
                                     if type(r) is float else str(r), result.values())) +
                      ' \\\\ \\hline\n')
        out.write('\t\\end{tabular}\n')

        # tail
        out.write('\t\\caption{' + caption + '}\n')
        out.write('\\end{table}\n')


In [ ]:
def save_results_to_tbl_pm(results, sizes, fn, caption):
    with codecs.open(f'tbl/{fn}.tex', 'w', 'utf-8') as out:
        # head
        out.write('\\begin{table}[H]\n')
        out.write('\t\\centering\n')

        # body
        end = ' \\\\ \\hline\n'
        out.write('\t\\begin{tabular}{|c||' + 'c|' * len(sizes) + '}' + '\\hline\n')
        out.write('\t\tРозмір задачі & ' + ' & '.join(map(str, sizes)) + ' \\\\ \\hline \\hline\n')
        for algo, result in results.items():
            out.write('\t\t' + algo + ' & ' +
                      ' & '.join(map(lambda r: f'{r[0]:.2f} $\\pm$ {r[1]:.2f}'
                                     if type(r[0]) is float else f'{r[0]} $\\pm$ {r[1]}', result.values())) +
                      ' \\\\ \\hline\n')
        out.write('\t\\end{tabular}\n')

        # tail
        out.write('\t\\caption{' + caption + '}\n')
        out.write('\\end{table}\n')


In [ ]:
def save_results_to_img(results, sizes, fn, styles, title):
    plt.xscale('log')
    plt.yscale('log')
    for style, (algo, result) in zip(styles, results.items()):
        plt.plot(sizes, result.values(), style, label=algo)
    plt.xlabel('Розмір задачі', fontsize=16)
    plt.ylabel(title, fontsize=16)
    plt.legend(loc='best', fontsize=12)
    plt.grid(which='both')
    plt.savefig(f'img/{fn}.png', bbox_inches='tight')
    plt.show()


In [ ]:
def save_results_to_img_pm(results, sizes, fn, styles, title):
    plt.xscale('log')
    plt.yscale('log')
    for style, (algo, result) in zip(styles, results.items()):
        plt.plot(sizes, [r[0] for r in result.values()], style, label=algo)
        plt.fill_between(sizes,
                         [r[0] - r[1] / 5 for r in result.values()], 
                         [r[0] + r[1] / 5 for r in result.values()],
                         color=style[0], alpha=.1)
    plt.xlabel('Розмір задачі', fontsize=16)
    plt.ylabel(title, fontsize=16)
    plt.legend(loc='best', fontsize=12)
    plt.grid(which='both')
    plt.savefig(f'img/{fn}.png', bbox_inches='tight')
    plt.show()


In [ ]:
# pythonic way to implement generics (known as templates in C++)
from typing import Callable, TypeVar
T = TypeVar('T')

# Алгоритми

In [ ]:
def korpelevich(x_initial: T,
                lambda_: float,
                A: Callable[[T], T],
                ProjectionOntoC: Callable[[T], T],
                tolerance: float = 1e-5,
                max_iterations: int = 1e4,
                debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial

    while True:
        # step 1
        y_current = ProjectionOntoC(x_current - lambda_ * A(x_current))

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # step 2
        x_next = ProjectionOntoC(x_current - lambda_ * A(y_current))

        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_current = None


In [ ]:
def tseng(x_initial: T,
          lambda_: float,
          A: Callable[[T], T],
          ProjectionOntoC: Callable[[T], T],
          tolerance: float = 1e-5,
          max_iterations: int = 1e4,
          debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial

    while True:
        # step 1
        y_current = ProjectionOntoC(x_current - lambda_ * A(x_current))

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # step 2
        x_next = y_current - lambda_ * (A(y_current) - A(x_current))

        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_current = None


In [ ]:
def cached_tseng(x_initial: T,
                 lambda_: float,
                 A: Callable[[T], T],
                 ProjectionOntoC: Callable[[T], T],
                 tolerance: float = 1e-5,
                 max_iterations: int = 1e4,
                 debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial

    while True:
        # step 1
        A_x_current = A(x_current)
        y_current = ProjectionOntoC(x_current - lambda_ * A_x_current)

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # step 2
        x_next = y_current - lambda_ * (A(y_current) - A_x_current)

        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_current = None


In [ ]:
def popov(x_initial: T,
          y_initial: T,
          lambda_: float,
          A: Callable[[T], T],
          ProjectionOntoC: Callable[[T], T],
          tolerance: float = 1e-5,
          max_iterations: int = 1e4,
          debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial
    y_previous = y_initial

    while True:
        # step 1
        y_current = ProjectionOntoC(x_current - lambda_ * A(y_previous))
        
        # step 2
        x_next = ProjectionOntoC(x_current - lambda_ * A(y_current))

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance and
            np.linalg.norm(x_next - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_previous, y_current = y_current, None


In [ ]:
def cached_popov(x_initial: T,
                 y_initial: T,
                 lambda_: float,
                 A: Callable[[T], T],
                 ProjectionOntoC: Callable[[T], T],
                 tolerance: float = 1e-5,
                 max_iterations: int = 1e4,
                 debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial
    y_previous = y_initial
    A_y_previous, A_y_current = A(y_previous), None

    while True:
        # step 1
        y_current = ProjectionOntoC(x_current - lambda_ * A_y_previous)
        
        # step 2
        A_y_current = A(y_current)
        x_next = ProjectionOntoC(x_current - lambda_ * A_y_current)

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance and
            np.linalg.norm(x_next - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_previous, y_current = y_current, None
        A_y_previous, A_y_current = A_y_current, None


# Перша задача

In [ ]:
def generate_matrix(size: int) -> np.matrix:
    matrix = np.zeros((size, size))
    for row in range(size):
        col = size - 1 - row
        if col < row:
            matrix[row, col] = 1
        if col > row:
            matrix[row, col] = -1
    return matrix


In [ ]:
sizes = (1_000, 2_000, 5_000, 10_000)
algos = ('Корпелевич', 'Tseng', 'Кеш. Tseng', 'Попов', 'Кеш. Попов')
iters, times = {algo: {} for algo in algos}, {algo: {} for algo in algos}

In [ ]:
for size in sizes:
    print(f'Розмір задачі = {size}:')
    a = generate_matrix(size)

    print('\tКорпелевич:', end=' ')
    _, iters['Корпелевич'][size], times['Корпелевич'][size] = korpelevich(
        x_initial=np.ones(size), lambda_=0.4,
        A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
        tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tTseng:', end=' ')
    _, iters['Tseng'][size], times['Tseng'][size] = tseng(
        x_initial=np.ones(size), lambda_=0.4,
        A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
        tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tКеш. Tseng:', end=' ')
    _, iters['Кеш. Tseng'][size], times['Кеш. Tseng'][size] = cached_tseng(
        x_initial=np.ones(size), lambda_=0.4,
        A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
        tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tПопов:', end=' ')
    _, iters['Попов'][size], times['Попов'][size] = popov(
        x_initial=np.ones(size), y_initial=np.ones(size), lambda_=0.4,
        A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
        tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tКеш. Попов:', end=' ')
    _, iters['Кеш. Попов'][size], times['Кеш. Попов'][size] = cached_popov(
        x_initial=np.ones(size), y_initial=np.ones(size), lambda_=0.4,
        A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
        tolerance=1e-3, max_iterations=1e4, debug=True)

    print()
    

In [ ]:
save_results_to_tbl(times, sizes,
    fn='problem-1-time',
    caption='Час виконання, секунд')
save_results_to_tbl(iters, sizes,
    fn='problem-1-iter',
    caption='Число ітерацій')

In [ ]:
save_results_to_img(times, sizes,
    fn='problem-1-time',
    styles=('r-', 'g-', 'g--', 'b-', 'b--'),
    title='Час виконання, секунд')

# Розріджені матриці

In [ ]:
def generate_sparse_matrix(size: int) -> np.matrix:
    data, rows, cols = [], [], []
    for row in range(size):
        rows.append(row)
        col = size - 1 - row
        cols.append(col)
        if col < row:
            data.append(1)
        if col > row:
            data.append(-1)
    return sparse.coo_matrix((data, (rows, cols)), shape=(size, size))


In [ ]:
sizes = (50_000, 100_000, 200_000, 500_000)
algos = ('Корпелевич', 'Tseng', 'Кеш. Tseng', 'Попов', 'Кеш. Попов')
iters, times = {algo: {} for algo in algos}, {algo: {} for algo in algos}

In [ ]:
for size in sizes:
    print(f'Розмір задачі = {size}:')
    a = generate_sparse_matrix(size)

    print('\tКорпелевич:', end=' ')
    _, iters['Корпелевич'][size], times['Корпелевич'][size] = korpelevich(
        x_initial=np.ones(size), lambda_=0.4,
        A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
        tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tTseng:', end=' ')
    _, iters['Tseng'][size], times['Tseng'][size] = tseng(
        x_initial=np.ones(size), lambda_=0.4,
        A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
        tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tКеш. Tseng:', end=' ')
    _, iters['Кеш. Tseng'][size], times['Кеш. Tseng'][size] = cached_tseng(
        x_initial=np.ones(size), lambda_=0.4,
        A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
        tolerance=1e-3, max_iterations=1e4, debug=True)
    
    print('\tPopov:', end=' ')
    _, iters['Попов'][size], times['Попов'][size] = popov(
        x_initial=np.ones(size), y_initial=np.ones(size), lambda_=0.4,
        A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
        tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tКеш. Попов:', end=' ')
    _, iters['Кеш. Попов'][size], times['Кеш. Попов'][size] = cached_popov(
        x_initial=np.ones(size), y_initial=np.ones(size), lambda_=0.4,
        A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
        tolerance=1e-3, max_iterations=1e4, debug=True)
    
    print()


In [ ]:
save_results_to_tbl(times, sizes,
    fn='problem-1-sparse-time',
    caption='Час виконання, секунд')
save_results_to_tbl(iters, sizes,
    fn='problem-1-sparse-iter',
    caption='Число ітерацій')

In [ ]:
save_results_to_img(times, sizes,
    fn='problem-1-sparse-time',
    styles=('r-', 'g-', 'g--', 'b-', 'b--'),
    title='Час виконання, секунд')

# Друга задача

In [ ]:
def generate_random_matrix(size: int) -> np.matrix:
    a = np.random.uniform(-5, 5, (size, size))
    b = np.random.uniform(-5, 5, (size, size))
    # skew-symmetric
    for row in range(size):
        for col in range(row):
            b[col, row] = -b[row, col]
    d = np.diag(np.random.uniform(0, .3, size))
    q = np.random.uniform(-500, 0, size)
    return a.dot(a.T) + b + d, q


In [ ]:
def ProjectionOntoProbabilitySymplex(x: np.array) -> np.array:
    dimensionality = x.shape[0]
    x /= dimensionality
    sorted_x = np.flip(np.sort(x))
    prefix_sum = np.cumsum(sorted_x)
    to_compare = sorted_x + (1 - prefix_sum) / np.arange(1, dimensionality + 1)
    k = 0
    for j in range(1, dimensionality):
        if to_compare[j] > 0:
            k = j
    return dimensionality * np.maximum(np.zeros(dimensionality),
                                       x + (to_compare[k] - sorted_x[k]))

In [ ]:
sizes = (100, 200, 500, 1000)
algos = ('Корпелевич', 'Tseng', 'Кеш. Tseng', 'Попов', 'Кеш. Попов')
iters, times = {algo: {} for algo in algos}, {algo: {} for algo in algos}

In [ ]:
for size in sizes:
    print(f'Розмір задачі = {size}:')

    for algo in algos:
        iters[algo][size] = []
        times[algo][size] = []
 
    for _ in range(5):
        print(f'\tTry #{_ + 1}:')

        np.random.seed(_)
        M, q = generate_random_matrix(size)

        print('\t\tКорпелевич:', end=' ')
        _, iter_, time_ = korpelevich(
            x_initial=np.ones(size), lambda_=.4/np.linalg.norm(M),
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Корпелевич'][size].append(iter_)
        times['Корпелевич'][size].append(time_)

        print('\t\tTseng:', end=' ')
        _, iter_, time_ = tseng(
            x_initial=np.ones(size), lambda_=.4/np.linalg.norm(M),
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Tseng'][size].append(iter_)
        times['Tseng'][size].append(time_)

        print('\t\tКеш. Tseng:', end=' ')
        _, iter_, time_ = cached_tseng(
            x_initial=np.ones(size), lambda_=.4/np.linalg.norm(M),
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Кеш. Tseng'][size].append(iter_)
        times['Кеш. Tseng'][size].append(time_)

        print('\t\tПопов:', end=' ')
        _, iter_, time_ = popov(
            x_initial=np.ones(size), y_initial=np.ones(size), lambda_=.4/np.linalg.norm(M),
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Попов'][size].append(iter_)
        times['Попов'][size].append(time_)

        print('\t\tКеш. Попов:', end=' ')
        _, iter_, time_ = cached_popov(
            x_initial=np.ones(size), y_initial=np.ones(size), lambda_=.4/np.linalg.norm(M),
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Кеш. Попов'][size].append(iter_)
        times['Кеш. Попов'][size].append(time_)

        print()

    print()

In [ ]:
for algo in algos:
    for size in sizes:
        iters[algo][size] = (int(np.mean(iters[algo][size])), int(np.std(iters[algo][size])))
        times[algo][size] = (np.mean(times[algo][size]), np.std(times[algo][size]))

In [ ]:
save_results_to_tbl_pm(times, sizes,
    fn='problem-2-time',
    caption='Час виконання, секунд')
save_results_to_tbl_pm(iters, sizes,
    fn='problem-2-iter',
    caption='Число ітерацій')

In [ ]:
save_results_to_img_pm(times, sizes,
    fn='problem-2-time',
    styles=('r-', 'g-', 'g--', 'b-', 'b--'),
    title='Час виконання, секунд')

# Адаптивні алгоритми

In [ ]:
def adaptive_korpelevich(x_initial: T,
                         tau: float,
                         lambda_initial: float,
                         A: Callable[[T], T],
                         ProjectionOntoC: Callable[[T], T],
                         tolerance: float = 1e-5,
                         max_iterations: int = 1e4,
                         debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial
    lambda_current = lambda_initial

    while True:
        # step 1
        y_current = ProjectionOntoC(x_current - lambda_current * A(x_current))

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # step 2
        x_next = ProjectionOntoC(x_current - lambda_current * A(y_current))

        # step 3
        if (A(x_current) - A(y_current)).dot(x_next - y_current) <= 0:
            lambda_next = lambda_current
        else:
            lambda_next = min(lambda_current, tau / 2 * 
                (np.linalg.norm(x_current - y_current) ** 2 + 
                 np.linalg.norm(x_next - y_current) ** 2) /
                    (A(x_current) - A(y_current)).dot(x_next - y_current))
        
        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_current = None
        lambda_current, lambda_next = lambda_next, None


In [ ]:
def cached_adaptive_korpelevich(x_initial: T,
                                tau: float,
                                lambda_initial: float,
                                A: Callable[[T], T],
                                ProjectionOntoC: Callable[[T], T],
                                tolerance: float = 1e-5,
                                max_iterations: int = 1e4,
                                debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial
    lambda_current = lambda_initial
    A_x_current, A_y_current = None, None

    while True:
        # step 1
        A_x_current = A(x_current)
        y_current = ProjectionOntoC(x_current - lambda_current * A_x_current)

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # step 2
        A_y_current = A(y_current)
        x_next = ProjectionOntoC(x_current - lambda_current * A_y_current)

        # step 3
        product = (A_x_current - A_y_current).dot(x_next - y_current)
        if product <= 0:
            lambda_next = lambda_current
        else:
            lambda_next = min(lambda_current, tau / 2 *
                (np.linalg.norm(x_current - y_current) ** 2 + 
                 np.linalg.norm(x_next - y_current) ** 2) / product)
        
        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_current = None
        lambda_current, lambda_next = lambda_next, None
        A_x_current, A_y_current = None, None


In [ ]:
def adaptive_tseng(x_initial: T,
                   tau: float,
                   lambda_initial: float,
                   A: Callable[[T], T],
                   ProjectionOntoC: Callable[[T], T],
                   tolerance: float = 1e-5,
                   max_iterations: int = 1e4,
                   debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial
    lambda_current = lambda_initial

    while True:
        # step 1
        y_current = ProjectionOntoC(x_current - lambda_current * A(x_current))

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # step 2
        x_next = y_current - lambda_current * (A(y_current) - A(x_current))

        # step 3
        # if np.all(A(x_current) == A(y_current)):
        if np.linalg.norm(A(x_current) - A(y_current)) < tolerance:
            lambda_next = lambda_current
        else:
            lambda_next = min(lambda_current, tau *
                np.linalg.norm(x_current - y_current) /
                np.linalg.norm(A(x_current) - A(y_current)))
        
        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_current = None
        lambda_current, lambda_next = lambda_next, None


In [ ]:
def cached_adaptive_tseng(x_initial: T,
                          tau: float,
                          lambda_initial: float,
                          A: Callable[[T], T],
                          ProjectionOntoC: Callable[[T], T],
                          tolerance: float = 1e-5,
                          max_iterations: int = 1e4,
                          debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial
    lambda_current = lambda_initial
    A_x_current = None
    A_y_current = None

    while True:
        # step 1
        A_x_current = A(x_current)
        y_current = ProjectionOntoC(x_current - lambda_current * A_x_current)

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # step 2
        A_y_current = A(y_current)
        x_next = y_current - lambda_current * (A_y_current - A_x_current)

        # step 3
        # if np.all(A_x_current == A_y_current):
        if np.linalg.norm(A_x_current - A_y_current) < tolerance:
            lambda_next = lambda_current
        else:
            lambda_next = min(lambda_current, tau *
                np.linalg.norm(x_current - y_current) /
                np.linalg.norm(A_x_current - A_y_current))
        
        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_current = None
        lambda_current, lambda_next = lambda_next, None
        A_x_current, A_y_current = None, None


In [ ]:
def adaptive_popov(x_initial: T,
                   y_initial: T,
                   tau: float,
                   lambda_initial: float,
                   A: Callable[[T], T],
                   ProjectionOntoC: Callable[[T], T],
                   tolerance: float = 1e-5,
                   max_iterations: int = 1e4,
                   debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial
    y_previous = y_initial
    lambda_current = lambda_initial

    while True:
        # step 1
        y_current = ProjectionOntoC(x_current - lambda_current * A(y_previous))
        
        # step 2
        x_next = ProjectionOntoC(x_current - lambda_current * A(y_current))

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance and
            np.linalg.norm(x_next - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # step 3
        if (A(y_previous) - A(y_current)).dot(x_next - y_current) <= 0:
            lambda_next = lambda_current
        else:
            lambda_next = min(lambda_current, tau / 2 *
                (np.linalg.norm(y_previous - y_current) ** 2 +
                 np.linalg.norm(x_next - y_current) ** 2) /
                    (A(y_previous) - A(y_current)).dot(x_next - y_current))
        
        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_previous, y_current = y_current, None
        lambda_current, lambda_next = lambda_next, None


In [ ]:
def cached_adaptive_popov(x_initial: T,
                          y_initial: T,
                          tau: float,
                          lambda_initial: float,
                          A: Callable[[T], T],
                          ProjectionOntoC: Callable[[T], T],
                          tolerance: float = 1e-5,
                          max_iterations: int = 1e4,
                          debug: bool = False) -> T:
    start = time.time()

    # initialization
    iteration_n = 1
    x_current = x_initial
    y_previous = y_initial
    lambda_current = lambda_initial
    A_y_previous, A_y_current = A(y_previous), None

    while True:
        # step 1
        y_current = ProjectionOntoC(x_current - lambda_current * A_y_previous)
        
        # step 2
        A_y_current = A(y_current)
        x_next = ProjectionOntoC(x_current - lambda_current * A_y_current)

        # stopping criterion
        if (np.linalg.norm(x_current - y_current) < tolerance and
            np.linalg.norm(x_next - y_current) < tolerance or
            iteration_n == max_iterations):
            if debug:
                end = time.time()
                duration = end - start
                print(f'Took {iteration_n} iterations '
                      f'and {duration:.2f} seconds to converge.')
                return x_current, iteration_n, duration
            return x_current

        # step 3
        product = (A_y_previous - A_y_current).dot(x_next - y_current)
        if product <= 0:
            lambda_next = lambda_current
        else:
            lambda_next = min(lambda_current, tau / 2 *
                (np.linalg.norm(y_previous - y_current) ** 2 +
                 np.linalg.norm(x_next - y_current) ** 2) / product)
        
        # next iteration
        iteration_n += 1
        x_current, x_next = x_next, None
        y_previous, y_current = y_current, None
        lambda_current, lambda_next = lambda_next, None
        A_y_previous, A_y_current = A_y_current, None


In [ ]:
sizes = (1_000, 2_000, 5_000, 10_000)
algos = ('Адапт. Корпелевич', 'Кеш. адапт. Корпелевич',
         'Адапт. Tseng', 'Кеш. адапт. Tseng',
         'Адапт. Попов', 'Кеш. адапт. Попов')
iters, times = {algo: {} for algo in algos}, {algo: {} for algo in algos}

In [ ]:
for size in sizes:
    print(f'Розмір задачі = {size}:')
    a = generate_matrix(size)

    print('\tАдапт. Корпелевич:', end=' ')
    _, iters['Адапт. Корпелевич'][size], times['Адапт. Корпелевич'][size] = \
          adaptive_korpelevich(x_initial=np.ones(size), tau=.9, lambda_initial=1,
                               A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                               tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tКеш. адапт. Корпелевич:', end=' ')
    _, iters['Кеш. адапт. Корпелевич'][size], times['Кеш. адапт. Корпелевич'][size] = \
          cached_adaptive_korpelevich(x_initial=np.ones(size), tau=.9, lambda_initial=1,
                                      A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                                      tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tАдапт. Tseng:', end=' ')
    _, iters['Адапт. Tseng'][size], times['Адапт. Tseng'][size] = \
          adaptive_tseng(x_initial=np.ones(size), tau=.9, lambda_initial=1,
                         A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                         tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tКеш. адапт. Tseng:', end=' ')
    _, iters['Кеш. адапт. Tseng'][size], times['Кеш. адапт. Tseng'][size] = \
          cached_adaptive_tseng(x_initial=np.ones(size), tau=.9, lambda_initial=1,
                                A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                                tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tАдапт. Попов:', end=' ')
    _, iters['Адапт. Попов'][size], times['Адапт. Попов'][size] = \
          adaptive_popov(x_initial=np.ones(size), y_initial=np.ones(size),
                         tau=.3, lambda_initial=1,
                         A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                         tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tКеш. адапт. Попов:', end=' ')
    _, iters['Кеш. адапт. Попов'][size], times['Кеш. адапт. Попов'][size] = \
          cached_adaptive_popov(x_initial=np.ones(size), y_initial=np.ones(size),
                                tau=.3, lambda_initial=1,
                                A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                                tolerance=1e-3, max_iterations=1e4, debug=True)
    
    print()


In [ ]:
save_results_to_tbl(times, sizes,
    fn='problem-1-adapt-time',
    caption='Час виконання, секунд')
save_results_to_tbl(iters, sizes,
    fn='problem-1-adapt-iter',
    caption='Число ітерацій')

In [ ]:
save_results_to_img(times, sizes,
    fn='problem-1-adapt-time',
    styles=('r-', 'r--', 'g-', 'g--', 'b-', 'b--'),
    title='Час виконання, секунд')

In [ ]:
sizes = (50_000, 100_000, 200_000, 500_000)
algos = ('Адапт. Корпелевич', 'Кеш. адапт. Корпелевич',
         'Адапт. Tseng', 'Кеш. адапт. Tseng',
         'Адапт. Попов', 'Кеш. адапт. Попов')
iters, times = {algo: {} for algo in algos}, {algo: {} for algo in algos}

In [ ]:
for size in sizes:
    print(f'Розмір задачі = {size}:')
    a = generate_sparse_matrix(size)

    print('\tАдапт. Корпелевич:', end=' ')
    _, iters['Адапт. Корпелевич'][size], times['Адапт. Корпелевич'][size] = \
          adaptive_korpelevich(x_initial=np.ones(size), tau=.4, lambda_initial=1,
                               A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                               tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tКеш. адапт. Корпелевич:', end=' ')
    _, iters['Кеш. адапт. Корпелевич'][size], times['Кеш. адапт. Корпелевич'][size] = \
          cached_adaptive_korpelevich(x_initial=np.ones(size), tau=.4, lambda_initial=1,
                                      A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                                      tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tАдапт. Tseng:', end=' ')
    _, iters['Адапт. Tseng'][size], times['Адапт. Tseng'][size] = \
          adaptive_tseng(x_initial=np.ones(size), tau=.4, lambda_initial=1,
                         A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                         tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tКеш. адапт. Tseng:', end=' ')
    _, iters['Кеш. адапт. Tseng'][size], times['Кеш. адапт. Tseng'][size] = \
          cached_adaptive_tseng(x_initial=np.ones(size), tau=.4, lambda_initial=1,
                                A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                                tolerance=1e-3, max_iterations=1e4, debug=True)

    print('\tАдапт. Попов:', end=' ')
    _, iters['Адапт. Попов'][size], times['Адапт. Попов'][size] = \
          adaptive_popov(x_initial=np.ones(size), y_initial=np.ones(size),
                         tau=.4, lambda_initial=1,
                         A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                         tolerance=1e-3, max_iterations=1e4, debug=True)

    print("\tКеш. адапт. Попов: ", end="")
    _, iters['Кеш. адапт. Попов'][size], times['Кеш. адапт. Попов'][size] = \
          cached_adaptive_popov(x_initial=np.ones(size), y_initial=np.ones(size),
                                tau=.4, lambda_initial=1,
                                A=lambda x: a.dot(x), ProjectionOntoC=lambda x: x,
                                tolerance=1e-3, max_iterations=1e4, debug=True)
    
    print()


In [ ]:
save_results_to_tbl(times, sizes,
    fn='problem-1-sparse-adapt-time',
    caption='Час виконання, секунд')
save_results_to_tbl(iters, sizes,
    fn='problem-1-sparse-adapt-iter',
    caption='Число ітерацій')

In [ ]:
save_results_to_img(times, sizes,
    fn='problem-1-sparse-adapt-time',
    styles=('r-', 'r--', 'g-', 'g--', 'b-', 'b--'),
    title='Час виконання, секунд')

In [ ]:
sizes = (100, 200, 500, 1000)
algos = ('Адапт. Корпелевич', 'Кеш. адапт. Корпелевич',
         'Адапт. Tseng', 'Кеш. адапт. Tseng',
         'Адапт. Попов', 'Кеш. адапт. Попов')
iters, times = {algo: {} for algo in algos}, {algo: {} for algo in algos}

In [ ]:
for size in sizes:
    print(f'Розмір задачі = {size}:')

    for algo in algos:
        iters[algo][size] = []
        times[algo][size] = []
 
    for _ in range(5):
        print(f'\tTry #{_ + 1}:')

        np.random.seed(_)
        M, q = generate_random_matrix(size)
        
        print('\t\tАдапт. Корпелевич:', end=' ')
        _, iter_, time_ = adaptive_korpelevich(
            x_initial=np.ones(size), tau=.4, lambda_initial=1,
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Адапт. Корпелевич'][size].append(iter_)
        times['Адапт. Корпелевич'][size].append(time_)

        print('\t\tКеш. адапт. Корпелевич:', end=' ')
        _, iter_, time_ = cached_adaptive_korpelevich(
            x_initial=np.ones(size), tau=.4, lambda_initial=1,
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Кеш. адапт. Корпелевич'][size].append(iter_)
        times['Кеш. адапт. Корпелевич'][size].append(time_)

        print('\t\tАдапт. Tseng:', end=' ')
        _, iter_, time_ = adaptive_tseng(
            x_initial=np.ones(size), tau=.4, lambda_initial=1,
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Адапт. Tseng'][size].append(iter_)
        times['Адапт. Tseng'][size].append(time_)

        print('\t\tКеш. адапт. Tseng:', end=' ')
        _, iter_, time_ = cached_adaptive_tseng(
            x_initial=np.ones(size), tau=.4, lambda_initial=1,
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Кеш. адапт. Tseng'][size].append(iter_)
        times['Кеш. адапт. Tseng'][size].append(time_)

        print('\t\tАдапт. Попов:', end=' ')
        _, iter_, time_ = adaptive_popov(
            x_initial=np.ones(size), y_initial=np.ones(size), tau=.4, lambda_initial=1,
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Адапт. Попов'][size].append(iter_)
        times['Адапт. Попов'][size].append(time_)

        print('\t\tКеш. адапт. Попов:', end=' ')
        _, iter_, time_ = cached_adaptive_popov(
            x_initial=np.ones(size), y_initial=np.ones(size), tau=.4, lambda_initial=1,
            A=lambda x: M.dot(x) + q, ProjectionOntoC=ProjectionOntoProbabilitySymplex,
            tolerance=1e-3, max_iterations=1e4, debug=True)
        iters['Кеш. адапт. Попов'][size].append(iter_)
        times['Кеш. адапт. Попов'][size].append(time_)

        print()

    print()

In [ ]:
for algo in algos:
    for size in sizes:
        iters[algo][size] = (int(np.mean(iters[algo][size])), int(np.std(iters[algo][size])))
        times[algo][size] = (np.mean(times[algo][size]), np.std(times[algo][size]))

In [ ]:
save_results_to_tbl_pm(times, sizes,
    fn='problem-2-adapt-time',
    caption='Час виконання, секунд')
save_results_to_tbl_pm(iters, sizes,
    fn='problem-2-adapt-iter',
    caption='Число ітерацій')

In [ ]:
save_results_to_img_pm(times, sizes,
    fn='problem-2-adapt-time',
    styles=('r-', 'r--', 'g-', 'g--', 'b-', 'b--'),
    title='Час виконання, секунд')

# Задача 4

In [ ]:
def generate_tridiagonal_matrix(size: int) -> np.matrix:
    d = np.zeros((size, size))
    for row in range(size):
        d[row, row] = 4
        if row != 0:
            d[row, row - 1] = 1
        if row + 1 != size:
            d[row, row + 1] = -2
    c = -np.ones(size)
    return d, c

In [ ]:
sizes = (500, 1_000, 2_000, 5_000)
algos = ('Адапт. Корпелевич', 'Кеш. адапт. Корпелевич',
         'Адапт. Tseng', 'Кеш. адапт. Tseng',
         'Адапт. Попов', 'Кеш. адапт. Попов')
iters, times = {algo: {} for algo in algos}, {algo: {} for algo in algos}

In [ ]:
for size in sizes:
    print(f'Розмір задачі = {size}:')
    d, c = generate_tridiagonal_matrix(size)

    def f(x: np.array) -> np.array:
        y = np.hstack([[0], x, [0]])
        f1 = y[1:-1] ** 2 + y[:-2] ** 2 + y[1:-1] * y[:-2] + y[1:-1] * y[2:]
        return f1 + d.dot(x) + c

    print('\tАдапт. Корпелевич:', end=' ')
    _, iters['Адапт. Корпелевич'][size], times['Адапт. Корпелевич'][size] = \
          adaptive_korpelevich(x_initial=np.ones(size), tau=.4, lambda_initial=1,
                               A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                               tolerance=1e-6, max_iterations=1e4, debug=True)

    print('\tКеш. адапт. Корпелевич:', end=' ')
    _, iters['Кеш. адапт. Корпелевич'][size], times['Кеш. адапт. Корпелевич'][size] = \
          cached_adaptive_korpelevich(x_initial=np.ones(size), tau=.4, lambda_initial=1,
                                      A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                                      tolerance=1e-6, max_iterations=1e4, debug=True)

    print('\tАдапт. Tseng:', end=' ')
    _, iters['Адапт. Tseng'][size], times['Адапт. Tseng'][size] = \
          adaptive_tseng(x_initial=np.ones(size), tau=.4, lambda_initial=1,
                         A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                         tolerance=1e-6, max_iterations=1e4, debug=True)

    print('\tКеш. адапт. Tseng:', end=' ')
    _, iters['Кеш. адапт. Tseng'][size], times['Кеш. адапт. Tseng'][size] = \
          cached_adaptive_tseng(x_initial=np.ones(size), tau=.4, lambda_initial=1,
                                A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                                tolerance=1e-6, max_iterations=1e4, debug=True)

    print('\tАдапт. Попов:', end=' ')
    _, iters['Адапт. Попов'][size], times['Адапт. Попов'][size] = \
          adaptive_popov(x_initial=np.ones(size), y_initial=np.ones(size),
                         tau=.4, lambda_initial=1,
                         A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                         tolerance=1e-6, max_iterations=1e4, debug=True)

    print("\tКеш. адапт. Попов: ", end="")
    _, iters['Кеш. адапт. Попов'][size], times['Кеш. адапт. Попов'][size] = \
          cached_adaptive_popov(x_initial=np.ones(size), y_initial=np.ones(size),
                                tau=.4, lambda_initial=1,
                                A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                                tolerance=1e-6, max_iterations=1e4, debug=True)
    
    print()


In [ ]:
save_results_to_tbl(times, sizes,
    fn='problem-4-adapt-time',
    caption='Час виконання, секунд')
save_results_to_tbl(iters, sizes,
    fn='problem-4-adapt-iter',
    caption='Число ітерацій')

In [ ]:
save_results_to_img(times, sizes,
    fn='problem-4-adapt-time',
    styles=('r-', 'r--', 'g-', 'g--', 'b-', 'b--'),
    title='Час виконання, секунд')

In [ ]:
def generate_sparse_tridiagonal_matrix(size: int) -> np.matrix:
    data, rows, cols = [], [], []
    for row in range(size):
        if row != 0:
            rows.append(row)
            cols.append(row - 1)
            data.append(1)
        rows.append(row)
        cols.append(row)
        data.append(4)
        if row + 1 != size:
            rows.append(row)
            cols.append(row + 1)
            data.append(-2)
    d = sparse.coo_matrix((data, (rows, cols)), shape=(size, size))
    c = -np.ones(size)
    return d, c

In [ ]:
sizes = (20_000, 50_000, 100_000, 200_000)
algos = ('Адапт. Корпелевич', 'Кеш. адапт. Корпелевич',
         'Адапт. Tseng', 'Кеш. адапт. Tseng',
         'Адапт. Попов', 'Кеш. адапт. Попов')
iters, times = {algo: {} for algo in algos}, {algo: {} for algo in algos}

In [ ]:
for size in sizes:
    print(f'Розмір задачі = {size}:')
    d, c = generate_sparse_tridiagonal_matrix(size)

    def f(x: np.array) -> np.array:
        y = np.hstack([[0], x, [0]])
        f1 = y[1:-1] ** 2 + y[:-2] ** 2 + y[1:-1] * y[:-2] + y[1:-1] * y[2:]
        return f1 + d.dot(x) + c

    print('\tАдапт. Корпелевич:', end=' ')
    _, iters['Адапт. Корпелевич'][size], times['Адапт. Корпелевич'][size] = \
          adaptive_korpelevich(x_initial=np.zeros(size), tau=.4, lambda_initial=1,
                               A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                               tolerance=1e-6, max_iterations=1e4, debug=True)

    print('\tКеш. адапт. Корпелевич:', end=' ')
    _, iters['Кеш. адапт. Корпелевич'][size], times['Кеш. адапт. Корпелевич'][size] = \
          cached_adaptive_korpelevich(x_initial=np.zeros(size), tau=.4, lambda_initial=1,
                                      A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                                      tolerance=1e-6, max_iterations=1e4, debug=True)

    print('\tАдапт. Tseng:', end=' ')
    _, iters['Адапт. Tseng'][size], times['Адапт. Tseng'][size] = \
          adaptive_tseng(x_initial=np.zeros(size), tau=.4, lambda_initial=1,
                         A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                         tolerance=1e-6, max_iterations=1e4, debug=True)

    print('\tКеш. адапт. Tseng:', end=' ')
    _, iters['Кеш. адапт. Tseng'][size], times['Кеш. адапт. Tseng'][size] = \
          cached_adaptive_tseng(x_initial=np.zeros(size), tau=.4, lambda_initial=1,
                                A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                                tolerance=1e-6, max_iterations=1e4, debug=True)

    print('\tАдапт. Попов:', end=' ')
    _, iters['Адапт. Попов'][size], times['Адапт. Попов'][size] = \
          adaptive_popov(x_initial=np.zeros(size), y_initial=np.zeros(size),
                         tau=.4, lambda_initial=1,
                         A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                         tolerance=1e-6, max_iterations=1e4, debug=True)

    print("\tКеш. адапт. Попов: ", end="")
    _, iters['Кеш. адапт. Попов'][size], times['Кеш. адапт. Попов'][size] = \
          cached_adaptive_popov(x_initial=np.zeros(size), y_initial=np.zeros(size),
                                tau=.4, lambda_initial=1,
                                A=f, ProjectionOntoC=lambda x: np.maximum(x, np.zeros(x.shape[0])),
                                tolerance=1e-6, max_iterations=1e4, debug=True)
    
    print()


In [ ]:
save_results_to_tbl(times, sizes,
    fn='problem-4-sparse-adapt-time',
    caption='Час виконання, секунд')
save_results_to_tbl(iters, sizes,
    fn='problem-4-sparse-adapt-iter',
    caption='Число ітерацій')

In [ ]:
save_results_to_img(times, sizes,
    fn='problem-4-sparse-adapt-time',
    styles=('r-', 'r--', 'g-', 'g--', 'b-', 'b--'),
    title='Час виконання, секунд')